In [1]:
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1604941853369_0007,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-3>

In [2]:
from pyspark.sql.types import *
import pandas as pd
pd.options.display.max_columns = None
from pyspark.sql.functions import *

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<hr style="border:1px solid blue"></hr>

### LOADING DATA

#### Download Dataset & Load into HDFS

```
sudo su 
cd 
mkdir mov
cd mov

wget http://files.grouplens.org/datasets/movielens/ml-latest.zip
unzip ml-latest.zip
rm -f ml-latest.zip

hdfs dfs -mkdir /user/root/data/
hdfs dfs -mkdir /user/root/data/MOV/
hdfs dfs -mkdir /user/root/data/MOV/CSV

hdfs dfs -put ml-latest/* /user/root/data/MOV/CSV
```


#### Define schema for Movies Dataset


In [3]:
moviesStruct = [StructField("movieId", IntegerType(), True),
    StructField("title", StringType(), True),
    StructField("genres", StringType(), True)]

moviesSchema = StructType(moviesStruct)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Read Movies dataframe from HDFS

In [4]:
# Read movies from HDFS as CSV (FIRST TIME ONLY)

moviesDF = spark.read.format("csv") \
    .option("header", "true") \
    .option("delimiter", ",") \
    .schema(moviesSchema) \
    .load("hdfs:///user/root/data/MOV/CSV/movies.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Write Movies Dataframe to Parquet File

In [ ]:
moviesDF.write.parquet("hdfs:///user/root/data/MOV/PARQUET/movies.parquet")

#### Reload Movies Dataframe From Parquet File 

In [83]:
# (everytime after the first export to PARQUET)

moviesDF = spark.read.parquet("hdfs:///user/root/data/MOV/PARQUET/movies.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Make Use of Caching Features

In [84]:
# caching might be aof great help for crossvalidation -among others
# read caching comment for ratingsDF below

moviesDF.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[movieId: int, title: string, genres: string]

#### Define schema for Ratings Dataset

In [5]:
# Define schema for ratings dataset
ratingsStruct = [StructField("userId", IntegerType(), True),
    StructField("movieId", IntegerType(), True),
    StructField("rating", DoubleType(), True),
    StructField("timestamp", IntegerType(), True)]

ratingsSchema = StructType(ratingsStruct)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Read Ratings Dataframe from HDFS

In [6]:
# Read ratings from HDFS (FIRST TIME ONLY)
ratingsDF = spark.read.format("csv") \
    .option("header", "true") \
    .option("delimiter", ",") \
    .schema(ratingsSchema) \
    .load("hdfs:///user/root/data/MOV/CSV/ratings.csv")

ratingsDF.limit(10).toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   userId  movieId  rating   timestamp
0       1      307     3.5  1256677221
1       1      481     3.5  1256677456
2       1     1091     1.5  1256677471
3       1     1257     4.5  1256677460
4       1     1449     4.5  1256677264
5       1     1590     2.5  1256677236
6       1     1591     1.5  1256677475
7       1     2134     4.5  1256677464
8       1     2478     4.0  1256677239
9       1     2840     3.0  1256677500

#### Write Ratings Dataframe to Parquet File

In [ ]:
ratingsDF.write.parquet("hdfs:///user/root/data/MOV/PARQUET/ratings.parquet")

#### Reload Ratings Dataframe From Parquet File 

In [27]:
# LOAD RATINGS From Parquet File (everytime after the first export to PARQUEY)

ratingsDF = spark.read.parquet("hdfs:///user/root/data/MOV/PARQUET/ratings.parquet").drop("timestamp")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Make Use of Caching Features

In [28]:
# Caching might be of great help - especially for crossvalidation -among others
# it is recommended for RDD re-use in iterative machine learning applications
# Check the size of your data on disk, and the total memory available to spark
# to see how much of your data fits into memory
# If the RDD does not fit in memory, some partitions will not be cached and will be recomputed on the fly each time they're needed. 

ratingsDF.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[userId: int, movieId: int, rating: double]

<hr style="border:1px solid blue"></hr>

### Data Exploration & Filtering

#### Select Users Dataframe

In [15]:
# Unique Users Id :
usersDF = ratingsDF.select("userId").distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# Total User Count

usersDF.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

283228

In [17]:
usersDF.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['userId']

#### Group Rating Count by Users

In [20]:
ratingsDF.groupBy("userId").count().show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-----+
|userId|count|
+------+-----+
|148   |48   |
|496   |31   |
|833   |17   |
|1088  |5    |
|1342  |3    |
|1591  |52   |
|2122  |88   |
|2366  |25   |
|3918  |49   |
|4101  |3    |
+------+-----+
only showing top 10 rows

In [ ]:
# Summary 
# Got 1000209 ratings from 6040 users on 3883 movies.
print("Got {} ratings from {} users on {} movies.".format(ratingsDF.count(), usersDF.count(), moviesDF.count()))

In [ ]:
ratingsDF.columns

#### Select Max User ID 
This will be useful for user incrementation

In [ ]:
ratingsDF.agg({"userId": "max"}).collect()[0][0]

In [ ]:
ratingsDF.schema

#### Create Ratings View

In [ ]:
ratingsDF.createTempView("RATING")

#### Use SQL to select Ratings for a specific User

In [ ]:
%%sql
select * from RATING where userId = 283228

In [ ]:
%%sql
select count(*) from RATING where userId = 283228

#### Create a Sample Dataframe 
(example)

In [ ]:
sampleDF = moviesDF.sample(fraction=0.001)

<hr style="border:1px solid blue"></hr>

### Exploring Movies Dataset 

#### Select Comedy Movies Only

In [ ]:
moviesDF.filter(moviesDF.genres.contains('Comedy')).limit(10).toPandas()

#### Display the TOP rated Comedy Movies (not aggregated)


In [ ]:
moviesDF.filter(moviesDf.genres.contains('Comedy')) \
    .join(ratingsDF, "movieId") \
    .sort(col("rating").desc()) \
    .show(20, truncate=False)

#### Count of Comedy Movies Grouped By Rating 

In [ ]:
moviesDF.filter(moviesDF.genres.contains('Comedy')) \
    .join(ratingsDF, "movieId") \
    .groupBy(col("rating")).count().orderBy("rating") \
    .show(20, truncate=False)

#### MOST Rated Comedy Movies - No matter the rating

In [ ]:
moviesDF.filter(moviesDf.genres.contains('Comedy')) \
    .join(ratingsDF, "movieId") \
    .groupBy(col("movieId")).count().orderBy("count", ascending=False) \
    .show(20, truncate=False)

#### Most Rated Comedy Movies, grouped by Movie & Rating


In [ ]:
moviesDF.filter(moviesDf.genres.contains('Comedy')) \
    .join(ratingsDF, "movieId") \
    .groupBy(col("movieId"),col("title"), col("rating")).count().orderBy("count", ascending=False) \
    .show(20, truncate=False)

In [ ]:
from pyspark.sql.functions import *

#### Top Rated Comedy Movies with most ratings

In [ ]:
genreMovieDF = moviesDF.filter(moviesDF.genres.contains(genre))
    
moviesByRating_counts = genreMovieDF \
        .join(ratingsDF, "movieId") \
        .groupBy("movieId").count().alias("count").orderBy(desc("count"))
        #.groupBy("movieId").count().alias("ratings count").orderBy(desc("count"))

moviesByRating_Full = genreMovieDF.join(moviesByRating_counts, "movieId") \
                                  .dropDuplicates().orderBy(desc("count"))
    
movieByRating_full.show(20)

In [ ]:
movieByRating_Full.count()

In [ ]:
movieByRating_Full.coalesce(1) \
      .write \
      .option("header","true") \
      .option("sep",",") \
      .mode("overwrite") \
      .csv("file:///path/output/file") \

#### Get User Input

In [ ]:
# This Cell does not work on Jupyter unfortunately 
# due to Pyspark limited Python libraries integration
# This need py4j integration and a call to scala/java
# code for that is provided as a standalone pyton program


for row in sampleDf.rdd.collect():
    print("Please rate the following movie (1-5 (best), or 0 if not seen):\n" + row.title + ":")
    rate = int(input())

In [ ]:
joinDf = moviesDf.join(ratingsDf, "movieId")
joinDf.show(10, False)

In [ ]:
ratingsDF.distinct().groupBy("rating").count()

<hr style="border:1px solid blue"></hr>

### MODELING PART

### Model Training

### Splitting data into training & test sets

In [7]:

trainingDF,testDF = ratingsDF.randomSplit([0.8, 0.2], seed=12345)

# Got 1000209 ratings from 6040 users on 3883 movies.
#print("Training {}, test {}.".format(trainingDF.count(), testDF.count()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Timing Utility

In [11]:
from time import time
from datetime import timedelta

class T():
    def __enter__(self):
        self.start = time()
    def __exit__(self, type, value, traceback):
        self.end = time()
        elapsed = self.end - self.start
        print(str(timedelta(seconds=elapsed)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
trainingDF.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[userId: int, movieId: int, rating: double, timestamp: int]

In [9]:
testDF.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[userId: int, movieId: int, rating: double, timestamp: int]

In [41]:
trainingDF.select("userId").distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

281954

In [42]:
testDF.select("userId").distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

264083

At this point it is interesting to see that there might be users on the test dataset  
that have no occurence in the training dataset, as the figures below can possibly suggest

### Training ALS model on the data


In [10]:

from pyspark.ml.recommendation import ALS

als = ALS(maxIter=5,
          regParam=0.01, 
          implicitPrefs=False, 
          userCol="userId", 
          itemCol="movieId", 
          ratingCol="rating", 
          coldStartStrategy="drop")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Fit Model (just a test - DO NOT RUN if you still need to cross-validate)

with T():
    model = als.fit(trainingDF)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0:01:02.987745

<hr style="border:1px solid blue"></hr>

### Cross Validation & Model Selection

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# WARNING : THIS CAN ESCALATE VERY QUICKLY, EVEN WITH ONLY TWO MODELS TO BE TESTED
# UNLESS YOU ARE ABLE TO NARROW DOWN AND LOWER ITERATIONS CONSIDERABLY, 
# YOU WOULD BETTER AVOID THIS STEP ALL TOGETHER

# NOTE THAT USING AN RMSE MINIMIZING LOOP CAN ALSO DO THE JOB

#param_grid = ParamGridBuilder() \
#            .addGrid(als.rank, [10, 50, 100, 150]) \
#            .addGrid(als.regParam, [.01, .05, .1, .15]) \
#            .build()

#param_grid = ParamGridBuilder() \
#            .addGrid(als.rank, [50, 100]) \
#            .addGrid(als.regParam, [.05, .1]) \
#            .build()

param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [50]) \
            .addGrid(als.regParam, [.05]) \
            .build()

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(
           metricName="rmse", 
           labelCol="rating", 
           predictionCol="prediction") 

print ("Num models to be tested: ", len(param_grid))

In [ ]:
cv = CrossValidator(estimator=als, \
                    estimatorParamMaps=param_grid, \
                    evaluator=evaluator, \
                    numFolds=5
                    parallelism=4) # this last parallelism param is crucial for increasing performance

### Fit The Cross Validator 
#### *(Optional - VERY Time Consuming)*

In [ ]:

# WARNING : Running this from a notebook hooked to Spark through Livy 
# will end up in a timeout after waiting for a long time
# the job will still be running but it will be abruptly be killed due to a timeout setting
# you need to setup livy.server.session.timeout and extend it to 1h  in livy.conf on the server, and restart livy
# livy.server.session.state-retain.sec = 600s => move to 3600s 
# location : /etc/livy/conf/livy.conf

# check spark web UI
# check generated DAGs
# check memory usage

cvModel = cv.fit(trainingDF)

<hr style="border:1px solid blue"></hr>

### Model Evaluation

In [35]:
# Evaluate the model by computing the RMSE on the Rating Predictions established for test data

predictions = model.transform(testDF)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
predictions.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['userId', 'movieId', 'rating', 'prediction']

In [ ]:
trainingDF.filter(testDF.userId == "12").show()

In [39]:
testDF.filter(testDF.userId == "12").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|    12|     48|   2.5|
+------+-------+------+

In [ ]:
testDF.show(10, False)

In [ ]:
testDF.groupBy("userId").count().show(10, False)

In [38]:
with T():
    predictions.show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|12    |48     |2.5   |1.1086082 |
|65    |1291   |5.0   |3.00573   |
|65    |5952   |5.0   |3.584456  |
|65    |54286  |3.0   |4.1214256 |
|76    |356    |5.0   |3.7495227 |
|76    |457    |4.0   |3.3255415 |
|76    |1240   |5.0   |3.1485655 |
|76    |2329   |4.0   |3.980468  |
|76    |2712   |1.0   |2.7749074 |
|76    |2763   |3.5   |2.5788736 |
+------+-------+------+----------+
only showing top 10 rows

0:00:12.380447

In [ ]:
predictions.groupBy("userId").count().show(10, False)

### Evaluate using Regression Evaluator

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

# Lower values of RMSE indicate better fit

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

<hr style="border:1px solid blue"></hr>

### Model Utilization

#### Generate recommendations for each user

In [ ]:
# CAUTION : This takes too much time to compute
# Use subsets instead (cell below)

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)

# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

#### Generate recommendations for a subset of user

In [59]:
# Generate top 10 movie recommendations for a specified set of users
users = ratingsDF.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)

# Generate top 10 user recommendations for a specified set of movies
movies = ratingsDF.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
userSubsetRecs.show(10, False)

In [64]:
userSubsetRecs.limit(10).toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   userId                                    recommendations
0    1311  [(160824, 12.564236640930176), (121650, 9.9915...
1    1306  [(160824, 15.04566478729248), (186347, 13.2301...
2    1774  [(115840, 15.921332359313965), (88816, 13.7468...

<hr style="border:1px solid blue"></hr>

### Data Formatting

In [75]:
# Get model recommendation for users based on their rating 
# You will notice that the Returned Dataframe has nested values 
# DataFrame[userId: int, recommendations: array<struct<movieId:int,rating:float>>]
# You will also notice that predicted ratings are all over the place
# This is not strictly an issue as it is not used "directly" and it can be ignored. 
# For more info see: https://stackoverflow.com/q/29051520/426332

recoDF = userSubsetRecs.limit(10)
#.toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Data Formatting & Join for staging & presentation

In [92]:

# Denormalize Recommendation Dataframe & explode the inner list so it can be selectable as a column   
# and filter out the rating colums as we do not need it

recoDF.select(col("userId"),explode(col("recommendations"))) \
      .select(col("userId"),col("col.movieId")) \
      .join(moviesDF,"movieId") \
      .orderBy("userId") \
      .select("userId", "movieId", "title") \
      .toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    userId  movieId                                              title
0      148   119165                                 Lunch Break (2008)
1      148   121650                          The Loyal 47 Ronin (1958)
2      148   129516                                      Poison (1951)
3      148   141532                                   Retrieval (2006)
4      148   154580                              Remote Control (1992)
5      148   154598                                  I, a Woman (1965)
6      148   159467                   Fifi Howls from Happiness (2013)
7      148   160824                  W.A.R.: Women Against Rape (1987)
8      148   189325                                         The Change
9      148   191203                                 Ô Jerusalem (2006)
10     496    90533                            Marseillaise, La (1938)
11     496   119165                                 Lunch Break (2008)
12     496   121650                          The Loyal 47 Ronin (1958)
13    

In [ ]:
movieSubSetRecs.show(10, False)

In [ ]:
from pyspark.sql.functions import col

# Recommendations for a specific user
aUserId = 12
recommandations = userRecs.filter(col("userId") == aUserId)
recommandations.show(1, False)

In [ ]:
from pyspark.sql.functions import explode

# Let's flatten the movie recommandations and look in detail
userRecommandations = recommandations.select(
  explode(col("recommendations").movieId).alias("movieId")
)

print("Recommandations for user {} :".format(aUserId))

moviesDf.join(userRecommandations, "movieId").show(10, False)

In [ ]:
# Ratings from the user

ratingsDF.filter(col("userId") == aUserId) \
    .join(moviesDf, "movieId") \
    .sort(col("rating").desc()) \
    .show(100, truncate=False)

<hr style="border:1px solid blue"></hr>

### Log Model Experiments with MLFlow


#### Make SURE to read carefully the MLFLOW REAMDE FILE first


In [13]:
import mlflow
import mlflow.spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
#not necessary as it defaults to /mlruns
# mlflow.set_tracking_uri("file:///path/to/mlruns")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
mlflow.set_tracking_uri("file:///mlruns")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
mlflow.start_run()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<ActiveRun: >

In [25]:
mlflow.spark.log_model(model, "ALSmodel_Lite8")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
print("Model saved in run %s" % mlflow.active_run().info.run_uuid)

In [26]:
mlflow.end_run()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**Run MLFlow UI to check your model**
```
mlflow ui --backend-store-uri /mlrun --host 0.0.0.0&
```

go to http://YOUR.IP.ADD.RESS:5000 :

### Serve Model with MLFlow

```console
# Serve & Curl Request Examples  
(Make SURE to read carefully the MLFLOW REAMDE FILE first)

mlflow models serve -m /mlruns/0/6c8050941d0744b8ac3652ff22d40983/artifacts/ALSmodel_Lite2 -h 0.0.0.0 --port 9999 --no-conda


curl -X POST localhost:9999/invocations -H 'Content-Type: application/json; format=pandas-split' -d '{"columns":["userId","movieId","rating"], "data":[[1311,144210,5,1604450652]]}'  

curl --request POST http://localhost:9999/invocations --header 'Content-Type: application/json; format=pandas-split' --data @mlf_data.json

curl -X POST http://localhost:9999/invocations -H 'Content-Type: application/json; format=pandas-split' -d @mlf_data.json
```

